You may find this series of notebooks at [databricks-solutions/realtime-rag-agents-databricks-youcom](https://github.com/databricks-solutions/realtime-rag-agents-databricks-youcom). For more information about this solution accelerator, visit the [blog post](https://you.com/articles/unlocking-real-time-intelligence-for-ai-agents-with-you.com-and-databricks).

In [0]:
%pip install -U -qqqq mlflow-skinny[databricks]>=3.1.0 langgraph==0.3.4 databricks-langchain databricks-agents uv gspread google-auth
dbutils.library.restartPython()

In [0]:
import pandas as pd
import pyspark.sql.functions as F
import io
import yaml

# Agent Configs

1. Create a [Connection](https://docs.databricks.com/aws/en/connect/) to access the You.com Search API. 

    * This connection definition assumes that You.com API key is persisted as a Databricks [Secret](https://docs.databricks.com/aws/en/security/secrets/).

2. Create a [UC Function](https://docs.databricks.com/aws/en/udf/unity-catalog) that makes an [http_request](https://docs.databricks.com/aws/en/sql/language-manual/functions/http_request) using the connection to perform search for user's query.


## 0. Initial Configurations

**USER TASK**: Fill in the command below with the `catalog` and `schema` to use for storing the UC Function and MLflow Evaluate Datasets.

In [0]:
%%writefile common.yaml

catalog: # TODO: Fill in with UC Catalog
schema: # TODO: Fill in with UC Schema
  
secret_scope: # TODO: Fill in with Secret Scope where You.com API key
secret_key: # TODO: Fill in with Secret Key for You.com API key

## 1. Load Configurations

In [0]:
cfg = yaml.safe_load(open("common.yaml"))

catalog = cfg.get("catalog", "main")
schema = cfg.get("schema", "default")

secret_scope = cfg.get("secret_scope", None)
secret_key = cfg.get("secret_key", None)

function_name = "you_com_search_function"

function_path = f"{catalog}.{schema}.{function_name}"

connection = "you_com_connection"

assert secret_scope is not None and secret_scope != "", "Missing secret scope configuration"
assert secret_key is not None and secret_key != "", "Missing secret key configuration"

## 2. Create Connection and UC Function

In [0]:
host = "https://api.ydc-index.io"
port = 443
api_path = "/v2/search"
sql_query = f"""CREATE CONNECTION IF NOT EXISTS {connection}
TYPE HTTP
OPTIONS (
  host '{host}',
  port {port},
  base_path '{api_path}',
  bearer_token secret('{secret_scope}', '{secret_key}')
)"""

spark.sql(sql_query)

In [0]:
sql_query = f"""CREATE OR REPLACE FUNCTION {function_path}(query STRING)
RETURNS STRING
COMMENT 'Search for information relative to the query'
RETURN (
  http_request(
    conn => "{connection}",
    method => "GET",
    path => concat('?query=', url_encode(query))
  ).text
)"""
print(sql_query)
spark.sql(sql_query)

## [OPTIONAL] 3. Test UC Function 

In [0]:
df = spark.createDataFrame(pd.DataFrame({"query": ["What is the capital of France?"]}))

display(df.withColumn("search_results", F.expr(f"{function_path}(query)")))